Unet model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def unet_model(input_size=(128, 128, 3), num_classes=1):
    """
    Create a U-Net model for image segmentation.

    Parameters:
    input_size (tuple): Shape of the input image (height, width, channels).
    num_classes (int): Number of classes for segmentation (e.g., 1 for binary segmentation).

    Returns:
    tf.keras.Model: The U-Net model.
    """

    inputs = layers.Input(input_size)

    # --- Encoder ---
    # Block 1
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    # Block 2
    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    # Block 3
    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    # Block 4
    conv4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    # --- Bottleneck ---
    conv5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(conv5)

    # --- Decoder ---
    # Up Block 1
    up6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv5)
    merge6 = layers.concatenate([conv4, up6], axis=3)
    conv6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(merge6)
    conv6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)

    # Up Block 2
    up7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    merge7 = layers.concatenate([conv3, up7], axis=3)
    conv7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(merge7)
    conv7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)

    # Up Block 3
    up8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    merge8 = layers.concatenate([conv2, up8], axis=3)
    conv8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(merge8)
    conv8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)

    # Up Block 4
    up9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    merge9 = layers.concatenate([conv1, up9], axis=3)
    conv9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(merge9)
    conv9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)

    # --- Output Layer ---
    if num_classes == 1:
        outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(conv9)  # Binary segmentation
    else:
        outputs = layers.Conv2D(num_classes, (1, 1), activation='softmax')(conv9)  # Multi-class segmentation

    return models.Model(inputs, outputs)

# Example usage:
model = unet_model(input_size=(128, 128, 3), num_classes=1)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 128, 128, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 128, 128, 64)   │          1,792 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 128, 128, 64)   │         36,928 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 64, 64, 64)     │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 64, 64, 128)    │         73,856 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 64, 64, 128)    │        147,584 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 32, 32, 128)    │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 32, 32, 256)    │        295,168 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 32, 32, 256)    │        590,080 │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 16, 16, 256)    │              0 │ conv2d_5[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 16, 16, 512)    │      1,180,160 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 16, 16, 512)    │      2,359,808 │ conv2d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_3           │ (None, 8, 8, 512)      │              0 │ conv2d_7[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 8, 8, 1024)     │      4,719,616 │ max_pooling2d_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)         │ (None, 8, 8, 1024)     │      9,438,208 │ conv2d_8[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose          │ (None, 16, 16, 512)    │      2,097,664 │ conv2d_9[0][0]         │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 16, 16, 1024)   │              0 │ conv2d_7[0][0],        │
│                      

 Total params: 31,031,745 (118.38 MB)

 Trainable params: 31,031,745 (118.38 MB)

 Non-trainable params: 0 (0.00 B)

Calculate convolution output


In [ ]:
def calculate_conv_output(n, p, f, s):
    """
    Calculate the output shape of a convolution operation.

    Parameters:
    n (int): Size of the input (e.g., width or height of the input image).
    p (int): Padding applied to the input.
    f (int): Size of the convolutional filter (kernel size).
    s (int): Stride of the convolution.

    Returns:
    int: Size of the output (width or height after the convolution).
    """
    return ((n + 2 * p - f) // s) + 1

# Example usage
n = 6  # Size of the input
p = 0  # Padding
f = 3  # Filter size
s = 1  # Stride

output_size = calculate_conv_output(n, p, f, s)
print(f"The output size after the convolution is: {output_size}")


Calculate convolution Params

In [ ]:
def calculate_conv_params(f, n_c_prev, n_f):
    """
    Calculate the number of parameters in a convolutional layer.

    Parameters:
    f (int): Size of the filter (kernel size, e.g., 3 for a 3x3 filter).
    n_c_prev (int): Number of channels in the previous layer.
    n_f (int): Number of filters in the current layer.

    Returns:
    int: Total number of parameters in the convolutional layer.
    """
    # The formula: (f^2 * n_c_prev + 1) * n_f
    return (f * f * n_c_prev + 1) * n_f

# Example usage
f = 3       # Filter size (e.g., 3x3 filter)
n_c_prev = 3  # Number of channels in the previous layer
n_f = 2       # Number of filters in the current layer

total_params = calculate_conv_params(f, n_c_prev, n_f)
print(f"The total number of parameters in the convolutional layer is: {total_params}")


Normal CNN network model

In [ ]:
# import libraries
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
# Define the input layer with a shape of 28x28 pixels and 1 color channel (grayscale image)
inputs = keras.Input(shape=(39, 39, 3))

# First convolutional layer
# - 32 filters, each of size 3x3
# - ReLU activation function for non-linearity
x = layers.Conv2D(filters=10, kernel_size=3, activation="relu")(inputs)

# Second convolutional layer
# - 64 filters, each of size 3x3
# - ReLU activation function
x = layers.Conv2D(filters=20, kernel_size=5, activation="relu", strides=2)(x)

# Third convolutional layer
# - 128 filters, each of size 3x3
# - ReLU activation function
x = layers.Conv2D(filters=40, kernel_size=5, activation="relu", strides=2)(x)

# Second pooling layer
# - Max pooling with a 2x2 pool size, further reducing spatial dimensions by half (from 10x10 to 5x5)
#x = layers.MaxPooling2D(pool_size=2)(x)

# Flatten the output of the last convolutional layer
# This converts the 3D feature map into a 1D vector to be fed into the dense layer
x = layers.Flatten()(x)

# Output layer
# - 10 units for 10 classes (e.g., digits 0–9 in a digit classification task)
# - Softmax activation function for multi-class classification
outputs = layers.Dense(10, activation="softmax")(x)

# Create the model
model_no_max_pool = keras.Model(inputs=inputs, outputs=outputs)

# Print a summary of the model's architecture
model_no_max_pool.summary()

Extracting Biases in Python Using TensorFlow/Keras

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import DepthwiseConv2D, Conv2D, Input

# Define a Depthwise Separable Convolution Layer
input_shape = (32, 32, 3)  # Example input shape
inputs = Input(shape=input_shape)

# Example depthwise separable convolution
depthwise_layer = DepthwiseConv2D(kernel_size=(3, 3), use_bias=True)
pointwise_layer = Conv2D(32, kernel_size=(1, 1), use_bias=True)

# Build the layers (this initializes weights and biases)
x = depthwise_layer(inputs)
x = pointwise_layer(x)

# Access the biases
depthwise_biases = depthwise_layer.bias.numpy()
pointwise_biases = pointwise_layer.bias.numpy()

# Print biases
print("Depthwise Biases:", depthwise_biases)
print("Pointwise Biases:", pointwise_biases)

Depthwise Biases: [0. 0. 0.]
Pointwise Biases: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In Python, the formulas for **precision** and **recall** can be derived from the confusion matrix. Here's how they are defined:

### Formula Explanation
1. **Precision**:  
   The fraction of true positive predictions out of all positive predictions made by the model.  
   \[
   \text{Precision} = \frac{\text{True Positives (TP)}}{\text{True Positives (TP)} + \text{False Positives (FP)}}
   \]

2. **Recall** (Sensitivity):  
   The fraction of true positive predictions out of all actual positive cases.  
   \[
   \text{Recall} = \frac{\text{True Positives (TP)}}{\text{True Positives (TP)} + \text{False Negatives (FN)}}
   \]



### Output
For the given confusion matrix:
\[
\begin{bmatrix}
50 & 10 \\
5  & 35 \\
\end{bmatrix}
\]
- **Precision**: \( \frac{50}{50 + 10} = 0.833 \)  
- **Recall**: \( \frac{50}{50 + 5} = 0.909 \)  

This implementation ensures the calculations are efficient and handles edge cases (e.g., division by zero).

In [2]:
def precision_recall(confusion_matrix):
    """
    Calculate precision and recall from a confusion matrix.

    Args:
    confusion_matrix (2x2 array): [[TP, FP],
                                   [FN, TN]]

    Returns:
    tuple: Precision and Recall values
    """
    # Extract values from confusion matrix
    TP = confusion_matrix[0][0]  # True Positives
    FP = confusion_matrix[0][1]  # False Positives
    FN = confusion_matrix[1][0]  # False Negatives

    # Calculate Precision
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0.0

    # Calculate Recall
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0.0

    return precision, recall

# Example usage
conf_matrix = [[50, 10],  # TP, FP
               [5,  35]]  # FN, TN

precision, recall = precision_recall(conf_matrix)
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")


Precision: 0.83
Recall: 0.91
